In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas
from sklearn.utils import shuffle
import gensim
import numpy as np

In [3]:
raw_data = pandas.DataFrame.from_csv(path='../data/people_attributes.csv', index_col=None)
raw_data = raw_data.set_index('name', drop=False)
raw_data.head()

,name,occupation,description,gender
name,,,,
Barack Obama,Barack Obama,Politician,44th President of the United States of America,male
Barbara Boxer,Barbara Boxer,Politician,American politician,female
Bill Clinton,Bill Clinton,Politician,42nd President of the United States,male
Carly Fiorina,Carly Fiorina,Politician,American corporate executive and politician,female
Clint Eastwood,Clint Eastwood,Actor,actor and director from the United States,male


In [4]:
raw_data['gender'].replace('female', 0, inplace=True)
raw_data['gender'].replace('male', 1, inplace=True)
raw_data.head()

,name,occupation,description,gender
name,,,,
Barack Obama,Barack Obama,Politician,44th President of the United States of America,1
Barbara Boxer,Barbara Boxer,Politician,American politician,0
Bill Clinton,Bill Clinton,Politician,42nd President of the United States,1
Carly Fiorina,Carly Fiorina,Politician,American corporate executive and politician,0
Clint Eastwood,Clint Eastwood,Actor,actor and director from the United States,1


In [5]:
name_and_gender = raw_data.drop(['occupation','description'], axis=1)
#name_and_gender = name_and_gender.reset_index(inplace=True)
name_and_gender.head()

,name,gender
name,,
Barack Obama,Barack Obama,1
Barbara Boxer,Barbara Boxer,0
Bill Clinton,Bill Clinton,1
Carly Fiorina,Carly Fiorina,0
Clint Eastwood,Clint Eastwood,1


In [6]:
name_and_gender = shuffle(name_and_gender)
name_and_gender.head()

,name,gender
name,,
Ashok Vemuri,Ashok Vemuri,1
Satya Prabhakar,Satya Prabhakar,1
David S. Taylor,David S. Taylor,1
Lisa Murkowski,Lisa Murkowski,0
Ben Affleck,Ben Affleck,1


In [7]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('../data/GoogleNews-vectors-negative300.bin', binary=True) 

In [132]:
name_and_gender.index.values

array(['J. Michael Pearson', 'Paul Hermelin', 'James J. Greco',
       'Tatsumi Kimishima', 'Daymond John', 'Jeb Bush', 'John McCain',
       'Morgan Freeman', 'Joe Donnelly', 'Vittorio Colao', 'Butch Otter',
       'Jean-Pascal Tricoire', 'Greg Abbott', 'Wilfred Kiboro',
       'Herbert Fisk Johnson III', 'Dick Durbin',
       'Grandhi Mallikarjuna Rao', 'Dianne Feinstein', 'Mitch McConnell',
       'Ralph Lauren', 'Pete Ricketts', 'William R. Klesse',
       'Chris DeWolfe', 'Bob Menendez', 'Kate Winslet', 'Klaus Kleinfeld',
       'Sandra Bullock', 'Indra Nooyi', 'Syed B. Ali', 'James Goodnight',
       'Stuart Gulliver', 'Tom Cotton', 'Shantanu Narayen',
       'Rex Tillerson', 'Barbara Boxer', 'Mark Wahlberg', 'Mike Crapo',
       'Brian Sandoval', 'Tom Cruise', 'Mike Huckabee', 'Barbara Mikulski',
       'Hugh Jackman', 'Richard B. Handler', 'Terry Gou',
       'Shelley Moore Capito', 'Orrin Hatch', 'Dennis Daugaard',
       'Benedict Cumberbatch', 'Bernie Sanders', 'Rich Lesser'

In [8]:
def _name_not_has_vec(name):
    try:
        word2vec.word_vec(name.replace(' ','_'))
        return False
    except:
        return True

In [9]:
truncated_name_and_gender = name_and_gender.drop([name for name in name_and_gender.index.values if _name_not_has_vec(name)])

In [10]:
truncated_name_and_gender.describe()

,gender
count,241.000000
mean,0.804979
std,0.397041
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [11]:
truncated_name_and_gender.head()

,name,gender
name,,
Ashok Vemuri,Ashok Vemuri,1
Satya Prabhakar,Satya Prabhakar,1
Lisa Murkowski,Lisa Murkowski,0
Ben Affleck,Ben Affleck,1
Orrin Hatch,Orrin Hatch,1


In [12]:
name_and_vector = truncated_name_and_gender.drop(['gender'], axis=1)
name_and_vector.head()

,name
name,
Ashok Vemuri,Ashok Vemuri
Satya Prabhakar,Satya Prabhakar
Lisa Murkowski,Lisa Murkowski
Ben Affleck,Ben Affleck
Orrin Hatch,Orrin Hatch


In [13]:
def _get_vector(row):
    return word2vec.word_vec(row.replace(' ','_')).flatten()

In [14]:
name_and_vector = name_and_vector.applymap(_get_vector)
name_and_vector.head()

,name
name,
Ashok Vemuri,"[-0.124023, 0.0986328, 0.0114136, 0.0576172, -..."
Satya Prabhakar,"[0.0639648, 0.0332031, -0.0834961, 0.0162354, ..."
Lisa Murkowski,"[-0.306641, -0.263672, -0.239258, 0.231445, -0..."
Ben Affleck,"[-0.0241699, 0.0913086, -0.310547, 0.198242, -..."
Orrin Hatch,"[0.0378418, 0.0874023, -0.0524902, 0.271484, -..."


In [15]:
name_and_vector.columns = ['vector']
name_and_vector.head()

,vector
name,
Ashok Vemuri,"[-0.124023, 0.0986328, 0.0114136, 0.0576172, -..."
Satya Prabhakar,"[0.0639648, 0.0332031, -0.0834961, 0.0162354, ..."
Lisa Murkowski,"[-0.306641, -0.263672, -0.239258, 0.231445, -0..."
Ben Affleck,"[-0.0241699, 0.0913086, -0.310547, 0.198242, -..."
Orrin Hatch,"[0.0378418, 0.0874023, -0.0524902, 0.271484, -..."


In [16]:
vectors = name_and_vector.vector.apply(pandas.Series)
vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
name,,,,,,,,,,,,,,,,,,,,,
Ashok Vemuri,-0.124023,0.098633,0.011414,0.057617,-0.031006,-0.112793,0.134766,-0.125000,0.013367,0.051514,...,-0.003662,0.165039,-0.063477,0.038086,0.046387,0.074707,0.074219,0.113281,-0.007996,-0.002060
Satya Prabhakar,0.063965,0.033203,-0.083496,0.016235,0.104980,-0.156250,0.033691,-0.017944,0.040283,0.034668,...,0.007111,-0.023071,-0.073242,0.019775,0.053711,-0.055908,0.072754,0.109863,0.048828,-0.041016
Lisa Murkowski,-0.306641,-0.263672,-0.239258,0.231445,-0.129883,0.083496,0.103027,0.223633,0.073730,0.227539,...,-0.136719,-0.332031,0.066895,-0.507812,0.419922,-0.235352,0.125000,0.175781,0.085449,0.349609
Ben Affleck,-0.024170,0.091309,-0.310547,0.198242,-0.306641,-0.054932,-0.289062,-0.496094,0.119629,0.125977,...,0.086426,0.189453,-0.100098,-0.093262,0.253906,-0.298828,-0.026855,-0.132812,-0.219727,0.027100
Orrin Hatch,0.037842,0.087402,-0.052490,0.271484,-0.168945,0.063965,0.182617,0.253906,-0.092285,0.032471,...,0.005798,0.001839,0.145508,-0.102051,0.022339,0.054443,-0.068359,-0.050293,0.083008,0.176758


In [17]:
vectors.values

array([[-0.12402344,  0.09863281,  0.01141357, ...,  0.11328125,
        -0.00799561, -0.00205994],
       [ 0.06396484,  0.03320312, -0.08349609, ...,  0.10986328,
         0.04882812, -0.04101562],
       [-0.30664062, -0.26367188, -0.23925781, ...,  0.17578125,
         0.08544922,  0.34960938],
       ..., 
       [-0.0246582 ,  0.38867188,  0.04711914, ...,  0.23144531,
         0.31445312, -0.23144531],
       [-0.25      ,  0.37109375, -0.16992188, ...,  0.19140625,
         0.30859375, -0.21386719],
       [-0.06542969,  0.16503906, -0.23242188, ...,  0.125     ,
        -0.16894531,  0.5234375 ]], dtype=float32)

In [18]:
genders = pandas.Series(truncated_name_and_gender['gender'])
just_binary_genders = np.array(genders)
just_binary_genders

array([1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0])

In [19]:
train_data = vectors[:200].values
train_labels = just_binary_genders[:200]
test_data = vectors[200:].values
test_labels = just_binary_genders[200:]
train_data.shape

(200, 300)

In [20]:
model = keras.models.Sequential([
    keras.layers.Dense(1, input_shape=(300,), activation='sigmoid'),
    ])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 301       
Total params: 301
Trainable params: 301
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

In [22]:
model.fit(train_data, train_labels, verbose=2, epochs=100, validation_data=(test_data, test_labels))

Train on 200 samples, validate on 41 samples
Epoch 1/100
2s - loss: 0.2315 - acc: 0.6250 - val_loss: 0.2288 - val_acc: 0.7073
Epoch 2/100
0s - loss: 0.2117 - acc: 0.7200 - val_loss: 0.2097 - val_acc: 0.8049
Epoch 3/100
0s - loss: 0.1940 - acc: 0.7650 - val_loss: 0.1937 - val_acc: 0.8049
Epoch 4/100
0s - loss: 0.1809 - acc: 0.7950 - val_loss: 0.1803 - val_acc: 0.7805
Epoch 5/100
0s - loss: 0.1696 - acc: 0.8050 - val_loss: 0.1705 - val_acc: 0.7805
Epoch 6/100
0s - loss: 0.1612 - acc: 0.8050 - val_loss: 0.1631 - val_acc: 0.7805
Epoch 7/100
0s - loss: 0.1544 - acc: 0.8050 - val_loss: 0.1572 - val_acc: 0.7805
Epoch 8/100
0s - loss: 0.1496 - acc: 0.8050 - val_loss: 0.1524 - val_acc: 0.7805
Epoch 9/100
0s - loss: 0.1455 - acc: 0.8050 - val_loss: 0.1484 - val_acc: 0.7805
Epoch 10/100
0s - loss: 0.1419 - acc: 0.8100 - val_loss: 0.1445 - val_acc: 0.7805
Epoch 11/100
0s - loss: 0.1386 - acc: 0.8100 - val_loss: 0.1413 - val_acc: 0.7805
Epoch 12/100
0s - loss: 0.1359 - acc: 0.8100 - val_loss: 0.138

In [23]:
name_and_vector.head()

,vector
name,
Ashok Vemuri,"[-0.124023, 0.0986328, 0.0114136, 0.0576172, -..."
Satya Prabhakar,"[0.0639648, 0.0332031, -0.0834961, 0.0162354, ..."
Lisa Murkowski,"[-0.306641, -0.263672, -0.239258, 0.231445, -0..."
Ben Affleck,"[-0.0241699, 0.0913086, -0.310547, 0.198242, -..."
Orrin Hatch,"[0.0378418, 0.0874023, -0.0524902, 0.271484, -..."


In [60]:
model.predict(vectors.iloc[0].values)

ValueError: Error when checking : expected dense_1_input to have shape (None, 300) but got array with shape (300, 1)

In [69]:
def predict_one(vec):
    return model.predict(np.array([vec]))

In [70]:
test_predict = name_and_vector[200:].vector.apply(predict_one)
test_predict

name
Donald Trump              [[0.953093]]
Lamar Alexander           [[0.931888]]
Mark Ruffalo              [[0.853989]]
Kerry Killinger           [[0.929111]]
Steve Odland              [[0.909877]]
Mike Crapo                [[0.859091]]
Jude Law                  [[0.735987]]
Mel Karmazin              [[0.953074]]
John Thune                 [[0.90291]]
Brian Krzanich            [[0.608313]]
Joe Manchin               [[0.857959]]
Barbara Mikulski          [[0.133742]]
Julia Roberts             [[0.316151]]
Amy Klobuchar             [[0.495746]]
Kirsten Gillibrand        [[0.538292]]
Mila Kunis                [[0.343925]]
Ryan Gosling              [[0.662597]]
Gina Raimondo             [[0.581526]]
Klaus Kleinfeld           [[0.987544]]
Chuck Schumer             [[0.830377]]
Larry Ellison             [[0.956021]]
Jim Risch                 [[0.918059]]
Barack Obama              [[0.802563]]
Bill Haslam               [[0.928334]]
Helge Lund                [[0.889088]]
Brian Sandoval      